In [1]:
import json
import requests
import re
import nltk
import os 
from dotenv import load_dotenv
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from rake_nltk import Rake
#nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('vader_lexicon')

In [2]:
def searchTweets(search_term:str):
    load_dotenv()
    url = "https://api.twitter.com/2/tweets/search/recent?max_results=25&expansions=referenced_tweets.id&tweet.fields=text&query=lang%3Aen%20"+search_term

    payload={}
    headers = {
    'Authorization': os.getenv('BEARER_TOKEN'),
    'Cookie': 'guest_id=v1%3A165292464833657250; guest_id_ads=v1%3A165292464833657250; guest_id_marketing=v1%3A165292464833657250; personalization_id="v1_D50leSEsdlQN9nTvwQ6B+g=="'
    }
    response = requests.request("GET", url, headers=headers, data=payload).json()['includes']['tweets']

    tweets = [i['text'] for i in response]


    out_file = open("myfile.json", "w") 
    json.dump(tweets, out_file, indent = 6) 
    out_file.close() 
    
    with open('myfile.json') as f:
     data = json.load(f)

    return data

In [3]:
tweets = searchTweets("Biden")

In [4]:
print(tweets)

["FLASHBACK: Biden promised in 2020 he would 'never' use military as a 'prop' https://t.co/vgwVz4nUb0", 'Let me get this straight, the same people who applauded the “American carnage” speech at a freaking inauguration, out of all places and who defended the “good people on both sides“ speech, responding to Neo-Nazis…are hyperventilating over Biden’s speech and red background? GTFO.', 'The Biden administration is pivoting to a long-term strategy to assist Afghans who worked with or on behalf of the US government, according to a senior administration official.\n\nReported by @CNN \nhttps://t.co/t3BMWgvGVG https://t.co/9R5tKD04ij', 'Biden gave a unifying speech against fascism. If you think it was divisive, you might be a fascist.', 'The Biden Administration has announced John Podesta will be joining the White House as a Senior Advisor to the President', 'Biden called this a "peaceful protest." https://t.co/eRCxCG2Nd6', 'Joe Biden has an unhealthy obsession with Donald Trump.', 'I’m a Bla

In [5]:
#Build text pre-processing 
#would a class for the tweets be smart?

### Text Preprocessing

In [6]:

def process_text(text):
    stop_words = set(stopwords.words('english'))
    clean_text = re.sub(r'\b(https?://\S+)\b','',text) #remove URLs
    clean_text = " ".join(re.findall('\w{3,}',clean_text))
    word_tokens = word_tokenize(clean_text)
    filtered_text = " ".join([word for word in word_tokens if word not in stop_words])
    
    return filtered_text

In [7]:
clean_tweets = [process_text(i) for i in tweets]

In [8]:
#get keywords
clean_tweets

['FLASHBACK Biden promised 2020 would never use military prop',
 'Let get straight people applauded American carnage speech freaking inauguration places defended good people sides speech responding Neo Nazis hyperventilating Biden speech red background GTFO',
 'The Biden administration pivoting long term strategy assist Afghans worked behalf government according senior administration official Reported CNN',
 'Biden gave unifying speech fascism think divisive might fascist',
 'The Biden Administration announced John Podesta joining White House Senior Advisor President',
 'Biden called peaceful protest',
 'Joe Biden unhealthy obsession Donald Trump',
 'Black MAGA Republican Joe Biden knows NOTHING',
 'President Biden want clear Not every Republican even majority Republicans MAGA Republicans Not every Republican embraces extreme ideology MAGA Republicans Biden attacked smeared half country',
 'Angry man smears half people country supposed lead amp promised unite',
 'case know PedoHilter J

In [9]:
from statistics import mean
from math import floor

def get_keywords(sentences: list):
    '''
    Given a list of strings, return the top keywords that are at least 4 letters long
    '''        
    r = Rake()
    r.extract_keywords_from_sentences(sentences)
    r.get_ranked_phrases_with_scores()
    keywords = r.get_ranked_phrases()[0].split(" ")
    avg_keyword_len = floor(mean([len(i) for i in keywords]))
    best_keywords = [i for i in keywords if len(i)>avg_keyword_len and len(i)>3]
    return best_keywords

In [10]:
get_keywords(clean_tweets)


['straight',
 'applauded',
 'american',
 'carnage',
 'freaking',
 'inauguration',
 'defended',
 'responding',
 'hyperventilating',
 'background']

In [11]:
#get sentiment
from nltk.sentiment import SentimentIntensityAnalyzer
def get_avg_sentiment(tweets):
    sia = SentimentIntensityAnalyzer()

    term_sentiment = round(mean([sia.polarity_scores(tweet)['compound'] for tweet in tweets ]),2)

    if term_sentiment < -0.33:
        return ("negative", term_sentiment)
    elif term_sentiment > 0.33:
        return ("positive", term_sentiment)
    else:
        return ("neutral",term_sentiment)

get_avg_sentiment(clean_tweets)


('neutral', -0.12)

In [12]:
keywords = get_keywords(clean_tweets,)

tag=pos_tag(keywords)

[i[0] for i in tag if i[1] not in ['VBG','VBD','RB','PDT','POS','WDT','WP','WRB',]]

['applauded', 'american', 'carnage', 'inauguration', 'background']

In [64]:
def call_trends():
  url = "https://api.twitter.com/1.1/trends/place.json?id=23424977"
  headers = {
  'Authorization': os.getenv("BEARER_TOKEN"),
  'Cookie': 'guest_id=v1%3A165292464833657250; guest_id_ads=v1%3A165292464833657250; guest_id_marketing=v1%3A165292464833657250; personalization_id="v1_D50leSEsdlQN9nTvwQ6B+g=="'
  }

  try:
    response = requests.request("GET",url,headers=headers).json()[0]['trends']
    response.sort(key=lambda x:0 if x["tweet_volume"] is None else x["tweet_volume"],reverse=True)
    results = [trend['name'] for trend in response]
    return results
  except Exception as e:
    error_message = f"An error occured in call_trends(): {e}"
    return error_message

In [65]:
x = call_trends()
print(x)

['Classified', 'The 12', 'Marines', 'MAGA Republican', '43 EMPTY', 'Top 4', 'Labor Day', 'Notifications', 'CPAC', 'What Americans', '#CashAppExtraCredit', 'Tiffany Haddish', 'Gazprom', 'Nikki', 'Bill Barr', 'Notis', 'Aries Spears', '#DemsWillWin', 'College Football Playoff', 'Peraza', 'The CFP', 'Carti', '2seok', 'John Harwood', 'Podesta', 'Barbara Ehrenreich', '#FreeCodeFridayContest', 'Dimed', '#PMSFeelGoodFriday', '#MAGAIsOver', 'Le Creuset', 'Hicks and IKF', 'Mervo', 'Plesac', 'Steely Dan', 'March Madness', 'Splitgate', 'Joey Votto', 'Andy Murray', 'IKF and Hicks', 'Ted Phillips', 'Vanoss', 'Jane Fonda', 'Berrettini', 'Samsung Galaxy Watch5', 'Jalen Thompson', 'Group of 5', 'kian', 'Chris Licht', 'John Malone']
